# Edge Search

Notebook to search for extreme heat events that overlap years.
Fix to resolve problem TBD.

By Cascade Tuholske, 2019-10-19


### Preliminary Findings

- In the entire record, there are 97 events that start on Jan 1.
- In the entire record, there are 94 events that end of Dec 31.

Of these, it looks like 5 were from the same city and bridged two years

#### Depdencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
fn = "/home/cascade/projects/data_out_urbanheat/All_data20191107.csv"  # Note: Need ?dl=1 to make sure this file gets read correctly
df = pd.read_csv(fn)
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Event_ID', 'ID_HDC_G0', 'CTR_MN_NM',
       'total_days', 'duration', 'avg_temp', 'avg_intensity', 'tot_intensity',
       'event_dates', 'intensity', 'tmax', 'year', 'region', 'sub-region',
       'intermediate-region', 'GCPNT_LAT', 'GCPNT_LON'],
      dtype='object')

## 1. Find Edges

Build query to find dates XXXX.12.31 and XXXX.01.01 
Events col are strings

In [80]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Event_ID', 'ID_HDC_G0', 'CTR_MN_NM',
       'total_days', 'duration', 'avg_temp', 'avg_intensity', 'tot_intensity',
       'event_dates', 'intensity', 'tmax', 'year', 'region', 'sub-region',
       'intermediate-region', 'GCPNT_LAT', 'GCPNT_LON'],
      dtype='object')

In [81]:
def date_search(df, date):
    
    """ Searches Tmax data frame to find dates within a Tmax event with the goal of finding 12.31-01.01 overlap
    Args:
        df = tmax df
        data = date you want to find
    
    Returns df with event id, event dates, city id, year, and tmax temps 
    """

    event_id_list = []
    event_dates_list = []
    city_id_list = []
    event_year_list = []
    tmax_list = []
    total_days_list = []
    
    for index, row in df.iterrows():
        if date in row['event_dates']:
            
            event_id = row['Event_ID']
            event_dates = row['event_dates']
            city_id = row['ID_HDC_G0']
            event_year = row['year']
            tmax = row['tmax']
            total_days = row['total_days']
            
            event_id_list.append(event_id)
            event_dates_list.append(event_dates)
            city_id_list.append(city_id)
            event_year_list.append(event_year)
            tmax_list.append(tmax)
            total_days_list.append(total_days)
    
    df_out = pd.DataFrame()
    df_out['ID_HDC_G0'] = city_id_list
    df_out['Event_ID'] = event_id_list
    df_out['tmax'] = tmax_list
    df_out['event_dates'] = event_dates_list
    df_out['year'] = event_year_list
    df_out['total_days'] = total_days_list
    
    return df_out

In [82]:
# Dec 31 Events

df_1231 = date_search(df, '12.31')

In [83]:
# Jan 1 Events

df_0101 = date_search(df, '01.01')

In [84]:
df_1231.head()

,ID_HDC_G0,Event_ID,tmax,event_dates,year,total_days
0,1506,633720,[40.846912],['2008.12.31'],2008,70
1,5519,117936,[40.956707 41.224346 41.3726 ],['1987.12.29' '1987.12.30' '1987.12.31'],1987,31
2,5519,849342,[41.66478],['2015.12.31'],2015,74
3,5551,117949,[40.935562 40.927567],['1987.12.30' '1987.12.31'],1987,12
4,5551,849363,[40.852245],['2015.12.31'],2015,31


In [85]:
# Check len

print(len(df_0101))
print(len(df_1231))

94
97


In [86]:
# See how many cities overlap

df_1231['ID_HDC_G0'].isin(df_0101['ID_HDC_G0']).value_counts()

True     59
False    38
Name: ID_HDC_G0, dtype: int64

In [87]:
# Merge based on city ID to only include overlaps

merge = pd.merge(df_1231, df_0101, on = 'ID_HDC_G0', how = 'inner')

In [88]:
merge

,ID_HDC_G0,Event_ID_x,tmax_x,event_dates_x,year_x,total_days_x,Event_ID_y,tmax_y,event_dates_y,year_y,total_days_y
0,5519,117936,[40.956707 41.224346 41.3726 ],['1987.12.29' '1987.12.30' '1987.12.31'],1987,31,451678,[40.648888],['2001.01.20'],2001,50
1,5519,849342,[41.66478],['2015.12.31'],2015,74,451678,[40.648888],['2001.01.20'],2001,50
2,5534,90648,[40.77078],['1986.12.31'],1986,86,45819,[41.395733 41.12193 ],['1984.01.01' '1984.01.02'],1984,103
3,5534,90648,[40.77078],['1986.12.31'],1986,86,141081,[41.37887 41.26748],['1988.01.01' '1988.01.02'],1988,84
4,5534,90648,[40.77078],['1986.12.31'],1986,86,404842,[41.404846 41.652008 41.585175 41.031097],['1999.01.01' '1999.01.02' '1999.01.03' '1999....,1999,92
...,...,...,...,...,...,...,...,...,...,...,...
314,3398,118141,[41.71855],['1987.12.31'],1987,8,68044,[42.620567 42.454918],['1985.01.01' '1985.01.02'],1985,5
315,3398,118141,[41.71855],['1987.12.31'],1987,8,141242,[43.16907 43.610813],['1988.01.01' '1988.01.02'],1988,7
316,3398,118141,[41.71855],['1987.12.31'],1987,8,451891,[40.78469],['2001.01.06'],2001,3
317,3398,118141,[41.71855],['1987.12.31'],1987,8,451892,[41.174187],['2001.01.08'],2001,3


In [89]:
merge.dtypes

ID_HDC_G0         int64
Event_ID_x        int64
tmax_x           object
event_dates_x    object
year_x            int64
total_days_x      int64
Event_ID_y        int64
tmax_y           object
event_dates_y    object
year_y            int64
total_days_y      int64
dtype: object

In [90]:
# Look for years the are one apart and get rows

out = []

for i, year in merge.iterrows():
        if year['year_y'] - year['year_x'] == 1:
            out.append(i)
out

[13, 34, 111, 123, 142, 175, 202, 236, 250, 273, 284, 299, 309, 315]

In [91]:
# Get the rows with dec 31 - jan 1

overlap = merge.loc[out]
overlap

,ID_HDC_G0,Event_ID_x,tmax_x,event_dates_x,year_x,total_days_x,Event_ID_y,tmax_y,event_dates_y,year_y,total_days_y
13,5534,117974,[41.446693 42.30564 42.97953 43.17466 ],['1987.12.28' '1987.12.29' '1987.12.30' '1987....,1987,99,141081,[41.37887 41.26748],['1988.01.01' '1988.01.02'],1988,84
34,5534,376959,[40.947575],['1998.12.31'],1998,79,404842,[41.404846 41.652008 41.585175 41.031097],['1999.01.01' '1999.01.02' '1999.01.03' '1999....,1999,92
111,5534,849422,[42.601044],['2015.12.31'],2015,126,882573,[41.114815 41.550606 41.746284],['2016.01.01' '2016.01.02' '2016.01.03'],2016,141
123,5541,117989,[41.22316 42.082108 42.755997 42.951126],['1987.12.28' '1987.12.29' '1987.12.30' '1987....,1987,92,141093,[41.167503 41.056114],['1988.01.01' '1988.01.02'],1988,75
142,5541,376974,[40.806267],['1998.12.31'],1998,76,404854,[41.231552 41.478714 41.41188 40.857803],['1999.01.01' '1999.01.02' '1999.01.03' '1999....,1999,84
175,5541,849443,[42.398857],['2015.12.31'],2015,112,882595,[40.954025 41.389816 41.585495],['2016.01.01' '2016.01.02' '2016.01.03'],2016,129
202,5520,118003,[41.090614 41.926125 42.600014 42.84202 ],['1987.12.28' '1987.12.29' '1987.12.30' '1987....,1987,83,141104,[40.96762 40.755005],['1988.01.01' '1988.01.02'],1988,70
236,5520,849463,[42.149605],['2015.12.31'],2015,103,882617,[40.83622 41.154823 41.233315],['2016.01.01' '2016.01.02' '2016.01.03'],2016,113
250,5390,377007,[40.932465],['1998.12.31'],1998,10,404896,[40.849594],['1999.01.01'],1999,33
273,5390,849508,[40.60667],['2015.12.31'],2015,59,882679,[41.256462 40.64724 ],['2016.01.01' '2016.01.02'],2016,71


## 2. Make new data from overlaps

In [92]:
def string_hunt(string_list, out_list, dtype):
    """Helper function to pull tmax record strings from a list without , and turn them into ints"""
    for i in string_list: # set the strings from X list
        if len(i) > 1:
            if '[' in i:
                
                record = i[1:]
                
                if ']' in record:
                    
                    record =  record[:-1]
                    out_list.append(dtype(record))
                else:
                    out_list.append(dtype(record))
            
            elif ']' in i:
                record = i[:-1]
                out_list.append(dtype(record))
            
            else:
                record = i
                out_list.append(dtype(record))
    
    return out_list

In [95]:
# loop by row to get temps

df_overlap = pd.DataFrame()

# Lists for df
temps_list_list = []
dates_list_list = []
duration_list = []
avg_temp_list = []
intensity_list = []
avg_intensity_list = []
tot_intensity_list = []
city_id_list = []
year_x_list = []
year_y_list = []
event_x_id_list = [] # <<<<<---- going to use the ID for the Dec date for now
event_y_id_list = [] # <<<<<---- going to use the ID for the Dec date for now
total_days_x_list = [] # total number of days added to first year
total_days_y_list = [] # total number of days subtracted first year

### Tempature
for i, row in overlap.iterrows():
    
    ### Temp and Days
    temps_list = [] # make list to populate
    
    temps_x = (row['tmax_x'].split(' ')) # split up the strings from X list
    temps_list = string_hunt(temps_x, temps_list, float)

    dur_x = len(temps_list) # duration first year 
    
    temps_y = (row['tmax_y'].split(' ')) # split up the strings from X list
    temps_list = string_hunt(temps_y, temps_list, float)

    dur_y = len(temps_list) - dur_x # duration second year
    
    temps_list_list.append(temps_list)
    
    ## Total Days
    total_days_x = row['total_days_x'] + dur_y
    total_days_y = row['total_days_y'] - dur_y
    
    total_days_x_list.append(total_days_x)
    total_days_y_list.append(total_days_y)
   
    ### Dates
    dates_list = [] # make list to populate
    
    dates_x = (row['event_dates_x'].split(' ')) # split up the strings from X list
    dates_list = string_hunt(dates_x, dates_list, str)
    
    dates_y = (row['event_dates_y'].split(' ')) # split up the strings from X list
    dates_list = string_hunt(dates_y, dates_list, str)
    
    dates_list_list.append(dates_list) # append list for df 
    
    ### Duration
    duration = len(temps_list)
    duration_list.append(duration)
    
    ### Intensity [x - 13 for x in a]
    intensity = [x - 40.6 for x in temps_list] # <<<<<<-------------------------- UPDATE AS NEEDED
    intensity_list.append(intensity)
    
    ### Avg_temp
    avg_temp = np.mean(temps_list)
    avg_temp_list.append(avg_temp)
    
    ### avg_intensity
    avg_intensity = np.mean(intensity)
    avg_intensity_list.append(avg_intensity)
    
    ### tot_intensity
    tot_intensity = np.sum(intensity)
    tot_intensity_list.append(tot_intensity)
    
    ### city_id & total days & year, etc
    city_id = row['ID_HDC_G0']
    city_id_list.append(city_id)
    
    ### Year
    year_x = row['year_x']
    year_x_list.append(year_x)
    
    year_y = row['year_y']
    year_y_list.append(year_y)
    
    ### event ID
    event_x_id = row['Event_ID_x']
    event_x_id_list.append(event_x_id)
    event_y_id = row['Event_ID_y']
    event_y_id_list.append(event_y_id)
    
    #avg_temp	avg_intensity	tot_intensity	event_dates	intensity

In [96]:
df_overlap['ID_HDC_G0'] = city_id_list
df_overlap['Event_ID_x'] = event_x_id_list
df_overlap['Event_ID_y'] = event_y_id_list
df_overlap['year_x'] = year_x_list
df_overlap['year_y'] = year_y_list
df_overlap['total_days_x'] = total_days_x_list
df_overlap['total_days_y'] = total_days_y_list
df_overlap['tmax'] = temps_list_list
df_overlap['event_dates'] = dates_list_list
df_overlap['duration'] = duration_list
df_overlap['avg_temp'] = avg_temp_list
df_overlap['intensity'] = intensity_list
df_overlap['tot_intensity'] = tot_intensity_list
df_overlap['avg_intensity'] = avg_intensity_list
df_overlap.head(1)



,ID_HDC_G0,Event_ID_x,Event_ID_y,year_x,year_y,total_days_x,total_days_y,tmax,event_dates,duration,avg_temp,intensity,tot_intensity,avg_intensity
0,5534,117974,141081,1987,1988,101,82,"[41.446693, 42.30564, 42.97953, 43.17466, 41.3...","['1987.12.28', '1987.12.29', '1987.12.30', '19...",6,42.092146,"[0.8466930000000019, 1.7056399999999954, 2.379...",8.952873,1.492145


## 3. Fix Total Days for Cities

Here we subtract the event days from the Jan year (y) from year y and we add those dates to year x so on balance the dates from the jan year are now just added to the earlier year

In [18]:
# Get List of Years and Cities for the dec-jan overlap and then find them in the dataset

# Start with year_x
years_x = list(df_out['year_x'])
id_x = list(df_out['ID_HDC_G0'])
total_days_x = list(df_out['total_days_x'])

x_list = []
for i in zip(years_x,id_x, total_days_x):
    x_list.append(i)

for x in x_list:
    print(x)

(1987, 5534, 101)
(1998, 5534, 83)
(2015, 5534, 129)
(1987, 5541, 94)
(1998, 5541, 80)
(2015, 5541, 115)
(1987, 5520, 85)
(2015, 5520, 106)
(1998, 5390, 11)
(2015, 5390, 61)
(1994, 1010, 17)
(2013, 1010, 30)
(1984, 3398, 18)
(1987, 3398, 10)


In [19]:
# Search df for i list and replace days
# this is super slow but it works

df_copy = df.copy()

for x in x_list:
    for i, row in df_copy.iterrows():
        if (row['year'] == x[0]) & (row['ID_HDC_G0'] == x[1]):
            print(df_copy.loc[i,'total_days'])
            df_copy.loc[i,'total_days'] = x[2]
            print(df_copy.loc[i,'total_days'])
            

99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
99
101
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
79
83
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
92
94
92
94
92
94
92
94
92
94
92
94
92
94
92
94
92
94
92
94
92
94
92
94
92
94
92
94
92
94
76
80
76
80
76
80
76
80
76
80
76
80
76
80
76
80
76
80
76
80
76
80
76
80
76
80
76
80
76
80
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
112
115
83
85
83
85
83
85
83
85
83
85
83
85
83
85
83
85
83
85
83
85
83
85
83
85
83
85
83
85
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
103
106
10
11
10
11
10
11
10
11
59
61
59
61
59
6

In [20]:
# Start with year_y 
years_y = list(df_out['year_y'])
id_y = list(df_out['ID_HDC_G0'])
total_days_y = list(df_out['total_days_y'])

y_list = []
for i in zip(years_y, id_y, total_days_y):
    y_list.append(i)

for y in y_list:
    print(y)

(1988, 5534, 82)
(1999, 5534, 88)
(2016, 5534, 138)
(1988, 5541, 73)
(1999, 5541, 80)
(2016, 5541, 126)
(1988, 5520, 68)
(2016, 5520, 110)
(1999, 5390, 32)
(2016, 5390, 69)
(1995, 1010, 11)
(2014, 1010, 10)
(1985, 3398, 3)
(1988, 3398, 5)


In [21]:
# Run on y_list

for y in y_list:
    for i, row in df_copy.iterrows():
        if (row['year'] == y[0]) & (row['ID_HDC_G0'] == y[1]):
            print(df_copy.loc[i,'total_days'])
            df_copy.loc[i,'total_days'] = y[2]
            print(df_copy.loc[i,'total_days'])

84
82
84
82
84
82
84
82
84
82
84
82
84
82
84
82
84
82
84
82
84
82
84
82
92
88
92
88
92
88
92
88
92
88
92
88
92
88
92
88
92
88
92
88
92
88
92
88
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
141
138
75
73
75
73
75
73
75
73
75
73
75
73
75
73
75
73
75
73
75
73
75
73
84
80
84
80
84
80
84
80
84
80
84
80
84
80
84
80
84
80
84
80
84
80
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
129
126
70
68
70
68
70
68
70
68
70
68
70
68
70
68
70
68
70
68
70
68
70
68
70
68
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
113
110
33
32
33
32
33
32
33
32
33
32
33
32
33
32
33
32
33
32
33
32
33
32
71
69
71
69
71
69
71
69
71
69
71
69
71
69
71
69
71
69
71
69
71
69
71
69
71
69
13
1

In [22]:
# Make a copy as back up in case you over right df_copy

df_copy_extra = df_copy.copy()

# 4. Add Meta data back

In [184]:
# copy df_out overlap, df_out is the overlap dataframe

df_overlap_copy = df_overlap.copy()

In [185]:
df_overlap_copy.head(1)

,ID_HDC_G0,Event_ID_x,Event_ID_y,year_x,year_y,total_days_x,total_days_y,tmax,event_dates,duration,avg_temp,intensity,tot_intensity,avg_intensity
0,5534,117974,141081,1987,1988,101,82,"[41.446693, 42.30564, 42.97953, 43.17466, 41.3...","['1987.12.28', '1987.12.29', '1987.12.30', '19...",6,42.092146,"[0.8466930000000019, 1.7056399999999954, 2.379...",8.952873,1.492145


In [186]:
# Get Columns to merge

cols_to_use = df.columns.difference(df_out_copy.columns) # find missing columns
cols_list = list(cols_to_use) # list
cols_list.append('ID_HDC_G0') # add IDS
df_cols = df_copy[cols_list]

In [188]:
# Drop duplicates

df_cols = df_cols.drop_duplicates('ID_HDC_G0', keep = 'first')

In [189]:
df_out_copy_merge = df_out_copy.merge(df_cols, on = 'ID_HDC_G0', how = 'inner')

In [190]:
df_out_copy_merge.columns

Index(['ID_HDC_G0', 'Event_ID_x', 'Event_ID_y', 'year', 'total_days', 'tmax',
       'event_dates', 'duration', 'avg_temp', 'intensity', 'tot_intensity',
       'avg_intensity', 'CTR_MN_NM', 'Event_ID', 'GCPNT_LAT', 'GCPNT_LON',
       'Unnamed: 0', 'Unnamed: 0.1', 'intermediate-region', 'region',
       'sub-region'],
      dtype='object')

In [191]:
# drop and rename columns

df_out_copy_merge = df_out_copy_merge.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])
df_out_copy.rename(columns = {'year_x':'year'}, inplace = True) 
df_out_copy.rename(columns = {'total_days_x':'total_days'}, inplace = True) 

# 5. Drop overlapped years and add in new DF

#### Drop overlap events based on event id

In [212]:
overlap.head(1)

,ID_HDC_G0,Event_ID_x,tmax_x,event_dates_x,year_x,total_days_x,Event_ID_y,tmax_y,event_dates_y,year_y,total_days_y
13,5534,117974,[41.446693 42.30564 42.97953 43.17466 ],['1987.12.28' '1987.12.29' '1987.12.30' '1987....,1987,99,141081,[41.37887 41.26748],['1988.01.01' '1988.01.02'],1988,84


In [213]:
# Get events

jan_ids = list(overlap['Event_ID_y'])
dec_ids = list(overlap['Event_ID_x'])

In [214]:
# Drop Events from Dataset

print(len(df_copy))
df_events = df_copy.copy()

# Jan
for event in jan_ids:
    df_events = df_events[df_events['Event_ID'] != event]
    
for event in dec_ids:
    df_events = df_events[df_events['Event_ID'] != event]

print(len(df_events))

883026
882998


#### Add in new events with new event ids

In [215]:
# Merge 
df_events = df_events.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
df_events.head()

,Event_ID,ID_HDC_G0,CTR_MN_NM,total_days,duration,avg_temp,avg_intensity,tot_intensity,event_dates,intensity,tmax,year,region,sub-region,intermediate-region,GCPNT_LAT,GCPNT_LON
0,1,2784,Germany,1,1,44.459750,3.859750,3.859750,['1983.07.27'],[3.85975],[44.45975],1983,Europe,Western Europe,Western Europe,48.138002,11.563435
1,763005,2784,Germany,1,1,40.658337,0.058337,0.058337,['2013.08.06'],[0.058337],[40.658337],2013,Europe,Western Europe,Western Europe,48.138002,11.563435
2,4,2833,Germany,1,1,47.493180,6.893180,6.893180,['1983.07.27'],[6.89318],[47.49318],1983,Europe,Western Europe,Western Europe,47.849642,12.102591
3,22703,2833,Germany,1,1,41.787350,1.187350,1.187350,['1984.07.11'],[1.18735],[41.78735],1984,Europe,Western Europe,Western Europe,47.849642,12.102591
4,349979,2833,Germany,1,1,41.393425,0.793425,0.793425,['1998.08.11'],[0.793425],[41.393425],1998,Europe,Western Europe,Western Europe,47.849642,12.102591


In [216]:
print(len(df_events))
print(len(df_out_copy_merge))

print(df_events.columns)
print(df_out_copy_merge.columns)

882998
14
Index(['Event_ID', 'ID_HDC_G0', 'CTR_MN_NM', 'total_days', 'duration',
       'avg_temp', 'avg_intensity', 'tot_intensity', 'event_dates',
       'intensity', 'tmax', 'year', 'region', 'sub-region',
       'intermediate-region', 'GCPNT_LAT', 'GCPNT_LON'],
      dtype='object')
Index(['ID_HDC_G0', 'Event_ID_x', 'Event_ID_y', 'year', 'total_days', 'tmax',
       'event_dates', 'duration', 'avg_temp', 'intensity', 'tot_intensity',
       'avg_intensity', 'CTR_MN_NM', 'Event_ID', 'GCPNT_LAT', 'GCPNT_LON',
       'intermediate-region', 'region', 'sub-region'],
      dtype='object')


In [218]:
# Make 'x' event ids for final df

df_out_copy_merge['Event_ID'] = df_out_copy_merge['Event_ID_x']

# drop event x y event ID cols 
df_out_copy_merge = df_out_copy_merge = df_out_copy_merge.drop(columns = ['Event_ID_x','Event_ID_y'])

In [219]:
df_out_copy_merge.head(1)

,ID_HDC_G0,year,total_days,tmax,event_dates,duration,avg_temp,intensity,tot_intensity,avg_intensity,CTR_MN_NM,Event_ID,GCPNT_LAT,GCPNT_LON,intermediate-region,region,sub-region
0,5534,1987,101,"[41.446693, 42.30564, 42.97953, 43.17466, 41.3...","['1987.12.28', '1987.12.29', '1987.12.30', '19...",6,42.092146,"[0.8466930000000019, 1.7056399999999954, 2.379...",8.952873,1.492145,Ethiopia,117974,4.177549,42.05758,Eastern Africa,Africa,Sub-Saharan Africa


In [220]:
print(len(df_events))
print(len(df_out_copy_merge))

df_final = pd.concat([df_events, df_out_copy_merge], sort = True)

print(len(df_final))

# print(len(df_out_copy_merge))
# print(len(df_events))
# df_copy = df_copy.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
# df_final = []
# df_final = pd.concat([df_copy, df_out_merge], sort = True)
# print(len(df_final))

882998
14
883012


In [221]:
# Save it out

# FN_OUT = "/home/cascade/projects/data_out_urbanheat/All_data20191107.csv"  # Note: Need ?dl=1 to make sure this file gets read correctly
# df_final.to_csv(FN_OUT)

### Sanity Check

In [205]:
for x in x_list:
    print(x)

(1987, 5534, 101)
(1998, 5534, 83)
(2015, 5534, 129)
(1987, 5541, 94)
(1998, 5541, 80)
(2015, 5541, 115)
(1987, 5520, 85)
(2015, 5520, 106)
(1998, 5390, 11)
(2015, 5390, 61)
(1994, 1010, 17)
(2013, 1010, 30)
(1984, 3398, 18)
(1987, 3398, 10)
(1987, 5534)
(1998, 5534)
(2015, 5534)
(1987, 5541)
(1998, 5541)
(2015, 5541)
(1987, 5520)
(2015, 5520)
(1998, 5390)
(2015, 5390)
(1994, 1010)
(2013, 1010)
(1984, 3398)
(1987, 3398)


In [211]:
for i, row in df_final[(df_final['year'] == 1987) & (df_final['ID_HDC_G0'] == 5534)].iterrows():
    print(row['total_days'])

101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101


In [210]:
for i, row in df[(df['year'] == 1987) & (df['ID_HDC_G0'] == 5534)].iterrows():
    print(row['total_days'])

99
99
99
99
99
99
99
99
99
99
99
99
99
99
99
99


# 6. Add start date

In [257]:
len(df_final)

883012

In [258]:
# Find First Data

test = df_final[0:10]

start_date_list = []
for i, row in df_final.iterrows():
    if type(row['event_dates']) is str:
        start_date = row['event_dates'].split('\'')[1]
        start_date_list.append(start_date)
    else:
        print(row['event_dates'][0])
        start_date = row['event_dates'][0]
        start_date_list.append(start_date)
#     else:
#         print(i)
#         print(row['event_dates'][0])
# #     print(row['event_dates'].split('\'')[1])
# #     dates = (row['event_dates'].split(' '))

'1987.12.28'
'1998.12.31'
'2015.12.31'
'1987.12.28'
'1998.12.31'
'2015.12.31'
'1987.12.28'
'2015.12.31'
'1998.12.31'
'2015.12.31'
'1994.12.30'
'2013.12.22'
'1984.12.30'
'1987.12.31'


In [259]:
len(start_date_list)

883012

In [260]:
df_final['start_date'] = start_date_list

In [263]:
FN_OUT = "/home/cascade/projects/data_out_urbanheat/All_data20191109_final.csv"  # Note: Need ?dl=1 to make sure this file gets read correctly
df_final.to_csv(FN_OUT)

## NEXT:

I need to write a function that compares years with the resulting subset of Dec 31 and Jan 1 and then I think we will just drop all these from the record and then add them on as heat waves that overlap years later. There should be about 100 of them

# Old Code

In [ ]:
            
            
    
#     temps_y = (row['tmax_y'].split(' ')) # split up the strings from Y list
            
#     for i in temps_y: # set the strings from Y list
#         if len(i) > 1:
#             if '[' in i:

#                 temp = i[1:]

#                 if ']' in temp:

#                     temp =  temp[:-1]
#                     temps_list.append(float(temp))
#                 else:
#                     temps_list.append(float(temp))

#             else:
#                 temp = i
#                 temps_list.append(float(temp))



# counter = 0
# for i, row in overlap.iterrows():
#     test = (row['tmax_x'].split(' '))
#     for i in test:
#         if len(i) > 1:
#             print(i)
            
#             counter = counter + 1
# #         if '[' in i:
# #             print('yes')
# #             print(i[1:])
# #         if ']' in i:
# #             print(i[:-1])
# #         print(i)
# print(counter)

In [ ]:

#     for i in temps_x: # set the strings from X list
#         if len(i) > 1:
#             if '[' in i:
                
#                 temp = i[1:]
                
#                 if ']' in temp:
                    
#                     temp =  temp[:-1]
#                     temps_list.append(float(temp))
#                 else:
#                     temps_list.append(float(temp))
            
#             elif ']' in i:
#                 temp = i[:-1]
#                 temps_list.append(float(temp))
            
#             else:
#                 temp = i
#                 temps_list.append(float(temp))
                
#             #temps_list_list.append(temps_list) # append list for df
    
#     temps_y = (row['tmax_y'].split(' ')) # split up the strings from X list

#     for i in temps_y: # set the strings from y list
#         if len(i) > 1:
#             if '[' in i:
                
#                 temp = i[1:]
                
#                 if ']' in temp:
                    
#                     temp =  temp[:-1]
#                     temps_list.append(float(temp))
#                 else:
#                     temps_list.append(float(temp))
            
#             elif ']' in i:
#                 temp = i[:-1]
#                 temps_list.append(float(temp))
            
#             else:
#                 temp = i
#                 temps_list.append(float(temp))
                       
#     temps_list_list.append(temps_list) # append list for df
    
#     print(temps_list)

In [ ]:
for i, row in overlap.iterrows():
    print(overlap.loc[i]['tmax_x'])

In [ ]:
[41.446693 42.30564 42.97953  43.17466
[40.947575]
[42.601044]
[41.22316 42.082108 42.755997 42.951126]
[40.806267]
[42.398857]
[41.090614 41.926125 42.600014 42.84202
[42.149605]
[40.932465]
[40.60667]
[41.324394 43.38537
[41.113922 41.99298 42.158752 43.608948 44.592743 42.73523 44.078674
45.146576 44.022064 41.813843]
[42.265797 43.199844]
[41.71855]

In [ ]:
            
#             print(df.loc[i,'year'])
#             print(df.loc[i,'total_days'])
            #print(row['year'], x[0], row['ID_HDC_G0'], x[1], row['total_days'], x[2])

# df_copy = df.copy()
# condition = (row['ID_HDC_G0'] == x[0]) #

# for i, row in df_sub.iterrows():
#     for x in x_list:
#         if condition:
#             print(row['total_days'], ' '+ x[2])

# for x in x_list:
#     print(df_sub.where((['year'] == x[0])))

# for i, row in df_copy.itter
# test = df[df['ID_HDC_G0'] == 5534]
# test = test[test['year'] == 1987]
# for i in zip(years_x,id_x, total_days_x):
#     print(i)

In [77]:
df_out_copy.head(1)

,ID_HDC_G0,Event_ID_x,Event_ID_y,year_x,year_y,total_days_x,total_days_y,tmax,event_dates,duration,avg_temp,intensity,tot_intensity,avg_intensity
0,5534,117974,141081,1987,1988,101,82,"[41.446693, 42.30564, 42.97953, 43.17466, 41.3...","['1987.12.28', '1987.12.29', '1987.12.30', '19...",6,42.092146,"[0.8466930000000019, 1.7056399999999954, 2.379...",8.952873,1.492145


In [78]:
df_out_copy = df_out_copy.drop(columns = ['year_y', 'total_days_y']) # drop ys


In [79]:
df_out_copy.head(1)

,ID_HDC_G0,Event_ID_x,Event_ID_y,year_x,total_days_x,tmax,event_dates,duration,avg_temp,intensity,tot_intensity,avg_intensity
0,5534,117974,141081,1987,101,"[41.446693, 42.30564, 42.97953, 43.17466, 41.3...","['1987.12.28', '1987.12.29', '1987.12.30', '19...",6,42.092146,"[0.8466930000000019, 1.7056399999999954, 2.379...",8.952873,1.492145


In [70]:
# Rename x columns

df_out_copy.rename(columns = {'year_x':'year'}, inplace = True) 
df_out_copy.rename(columns = {'total_days_x':'total_days'}, inplace = True) 

df_out_copy.head()

,ID_HDC_G0,Event_ID_x,Event_ID_y,year,total_days,tmax,event_dates,duration,avg_temp,intensity,tot_intensity,avg_intensity
0,5534,117974,141081,1987,101,"[41.446693, 42.30564, 42.97953, 43.17466, 41.3...","['1987.12.28', '1987.12.29', '1987.12.30', '19...",6,42.092146,"[0.8466930000000019, 1.7056399999999954, 2.379...",8.952873,1.492145
1,5534,376959,404842,1998,83,"[40.947575, 41.404846, 41.652008, 41.585175, 4...","['1998.12.31', '1999.01.01', '1999.01.02', '19...",5,41.324140,"[0.3475749999999991, 0.8048459999999977, 1.052...",3.620701,0.724140
2,5534,849422,882573,2015,129,"[42.601044, 41.114815, 41.550606, 41.746284]","['2015.12.31', '2016.01.01', '2016.01.02', '20...",4,41.753187,"[2.0010440000000003, 0.5148149999999987, 0.950...",4.612749,1.153187
3,5541,117989,141093,1987,94,"[41.22316, 42.082108, 42.755997, 42.951126, 41...","['1987.12.28', '1987.12.29', '1987.12.30', '19...",6,41.872668,"[0.6231599999999986, 1.4821079999999967, 2.155...",7.636008,1.272668
4,5541,376974,404854,1998,80,"[40.806267, 41.231552, 41.478714, 41.41188, 40...","['1998.12.31', '1999.01.01', '1999.01.02', '19...",5,41.157243,"[0.20626699999999687, 0.6315519999999992, 0.87...",2.786216,0.557243


In [62]:
# Merge in Missing colums

cols_to_use = df.columns.difference(df_out_copy.columns) # find missing columns
cols_list = list(cols_to_use) # list
cols_list.append('ID_HDC_G0') # add IDS

df_copy_cols = df_copy[cols_list] # .copy() # make a copy so you don't f it up
df_copy_cols = df_copy_cols.drop(columns=['Unnamed: 0', 'Unnamed: 0.1']) # remove weird cols

df_copy_cols = df_copy_cols.drop_duplicates('ID_HDC_G0', keep = 'first') # drop duplicates

In [63]:
# merge them in 
df_out_copy_merge = df_out_copy.merge(df_copy_cols, on = 'ID_HDC_G0', how = 'inner')

In [64]:
df_out_copy_merge.shape

(14, 19)

In [65]:
df_out_copy_merge.head(1)

,ID_HDC_G0,Event_ID_x,Event_ID_y,year,total_days,tmax,event_dates,duration,avg_temp,intensity,tot_intensity,avg_intensity,CTR_MN_NM,Event_ID,GCPNT_LAT,GCPNT_LON,intermediate-region,region,sub-region
0,5534,117974,141081,1987,101,"[41.446693, 42.30564, 42.97953, 43.17466, 41.3...","['1987.12.28', '1987.12.29', '1987.12.30', '19...",6,42.092146,"[0.8466930000000019, 1.7056399999999954, 2.379...",8.952873,1.492145,Ethiopia,22526,4.177549,42.05758,Eastern Africa,Africa,Sub-Saharan Africa


#### Drop overlap events based on event id

In [ ]:
# Get events

overlap
jan_ids = list(overlap['Event_ID_y'])
dec_ids = list(overlap['Event_ID_x'])

In [ ]:
# Drop Events from Dataset

print(len(df))
df_copy = df.copy()

# Jan
for event in jan_ids:
    df_copy = df_copy[df_copy['Event_ID'] != event]
    
for event in dec_ids:
    df_copy = df_copy[df_copy['Event_ID'] != event]

print(len(df_copy))

#### Add in new events with new event ids

In [ ]:
# Merge 
df_copy = df_copy.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
df_copy.head()

In [ ]:
print(len(df_copy))
print(len(df_out_merge))

print(df_copy.columns)
print(df_out_merge.columns)

In [ ]:
print(len(df_copy))
print(len(df_out))
df_copy = df_copy.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
df_final = []
df_final = pd.concat([df_copy, df_out_merge], sort = True)
print(len(df_final))

In [ ]:
df_final.head(50)